In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable as v
from tqdm import tqdm
import time 

is_gpu = torch.cuda.is_available()
if is_gpu:
    id = 0
    torch.cuda.set_device(id)
    
#gpu_nums = torch.cuda.device_count()
#gpu_index = torch.cuda.current_device()
#print(is_gpu,gpu_nums,gpu_index)
device = torch.device('cuda' if is_gpu else 'cpu')

#device = torch.device('cpu')
print(device)
torch.set_default_tensor_type('torch.DoubleTensor')
print(torch.__version__)

import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
 
def load_variable(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

cuda
1.13.0+cu117


In [14]:
int_sample_num = 50

theta = torch.linspace(2*np.pi/int_sample_num,2*np.pi,int_sample_num).reshape(-1,1)
r = 1 #9/20-1/9*torch.cos(5*theta)
x0,x1 = torch.cos(theta),torch.sin(theta)
int_x = torch.cat((x0,x1),1)*r
#rprime = torch.cat((-torch.sin(theta),torch.cos(theta)),1)
#int_n = rprime@torch.Tensor([[0,-1],[1,0]])
int_n = int_x/r#int_n/((int_n*int_n).sum(axis=1).reshape(-1,1).sqrt())          
mat = 0

if mat == 1:
    A2 = load_variable('./matrix/Poisson_Disc_1200.txt')
else:
    G2 = torch.zeros(int_sample_num,int_sample_num)
    for i in range(int_sample_num):
        d = int_x-int_x[i,:]
        r0 = d.norm(dim=1)
        G2[i,:] = -1/(2*np.pi)*((d*int_n).sum(axis=1))/(r0*r0)
    
            
    KaparR2 = torch.zeros(int_sample_num,1)
    KaparR2[1],KaparR2[2] = 1.825748064736159,-1.325748064736159
    A2 = torch.zeros(int_sample_num,int_sample_num)
    for i in range(int_sample_num):
        for j in range(int_sample_num):
            if i==j:
                A2[i,j] = 1/2
            else:
                A2[i,j] = -(1+KaparR2[min(abs(i-j), int_sample_num-abs(i-j))])*G2[i,j]/int_sample_num*2*np.pi
    #save_variable(A2,'./matrix/Poisson_Disc_1200.txt')



In [15]:
M0 = 160
r1 = torch.rand(M0,1).sqrt()
theta1 = torch.rand(M0,1)*2*np.pi
x0,x1 = torch.cos(theta1),torch.sin(theta1)
in_x = torch.cat((x0,x1),1)*r1
int_xy = torch.zeros(M0*int_sample_num,4)
for i in range(M0):
    int_xy[i*int_sample_num:(i+1)*int_sample_num,0:2] = in_x[i,:]
    int_xy[i*int_sample_num:(i+1)*int_sample_num,2:4] = int_x
sample_u = -(torch.log(((int_xy[:,0:2]-int_xy[:,2:4])**2).sum(axis=1).sqrt())/(2*np.pi)).reshape(-1,1)

In [5]:
m = 100
class myNet(nn.Module):
  def __init__(self):
    super(myNet, self).__init__()
    self.input = nn.Linear(4,m)
    self.block1=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block2=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block3=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block4=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block5=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block6=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block7=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block8=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    #self.block9=nn.Sequential(
    #  nn.Linear(m,m),nn.ReLU(),
    #  nn.Linear(m,m),nn.ReLU(),
    #)
    self.out = nn.Linear(m,1)
  def forward(self, x):
      x = self.input(x)
      x = self.block1(x) + x
      x = self.block2(x) + x
      x = self.block3(x) + x
      x = self.block4(x) + x
      x = self.block5(x) + x
      x = self.block6(x) + x
      x = self.block7(x) + x
      x = self.block8(x) + x
      #x = self.block9(x) + x
      x = self.out(x)
      return x
  
mynet1 = myNet() 


In [16]:
mynet1 = mynet1.to(device)
int_xy = int_xy.to(device)
A2 = A2.to(device)
sample_u = sample_u.to(device)

In [17]:

optimzer1 = torch.optim.Adam(mynet1.parameters(mynet1),lr=0.00001)
Epoc = 2000
loss_all = np.zeros(Epoc+1)
u0 = torch.zeros(M0*int_sample_num,1)
t = time.time()
for epoc in range(Epoc+1):
    loss = 0
    if (epoc-1)%200==0:
        r1 = torch.rand(M0,1).sqrt()
        theta1 = torch.rand(M0,1)*2*np.pi
        x0,x1 = torch.cos(theta1),torch.sin(theta1)
        in_x = torch.cat((x0,x1),1)*r1
        int_xy = torch.zeros(M0*int_sample_num,4)
        for i in range(M0):
            int_xy[i*int_sample_num:(i+1)*int_sample_num,0:2] = in_x[i,:]
            int_xy[i*int_sample_num:(i+1)*int_sample_num,2:4] = int_x
        sample_u = (torch.log(((int_xy[:,0:2]-int_xy[:,2:4])**2).sum(axis=1).sqrt())/(2*np.pi)).reshape(-1,1)
        sample_u = sample_u.to(device) 
        int_xy = int_xy.to(device)
        
    int_h = mynet1(int_xy)
    for i in range(M0):
        u0 = -A2@((int_h[i*int_sample_num:(i+1)*int_sample_num]))
        loss = loss + torch.mean(torch.pow(u0-sample_u[i*int_sample_num:(i+1)*int_sample_num],2))
    
    #loss = Green_loss_func(sample_u,u0)
    
    optimzer1.zero_grad()
    loss.backward(retain_graph=True)
    optimzer1.step()
    loss_all[epoc] = loss.cpu().detach().numpy() 
    if epoc%1000==0:
        print("%.5f"%loss_all[epoc],epoc,"%.4f"%(time.time()-t))
        t = time.time()
        #print((bd_u-u0).norm()**2/bd_sample_num)


5.48487 0 0.0594
0.00034 1000 48.7860
0.00010 2000 47.7498


In [21]:
mynet2 = mynet1.cpu()
torch.save(mynet2.state_dict(),'./Poisson_disc_50_160')

In [22]:

int_sample_num = 1200

theta = torch.linspace(2*np.pi/int_sample_num,2*np.pi,int_sample_num).reshape(-1,1)
r = 1 #9/20-1/9*torch.cos(5*theta)
x0,x1 = torch.cos(theta),torch.sin(theta)
int_x = torch.cat((x0,x1),1)*r
int_n = int_x/r#int_n/((int_n*int_n).sum(axis=1).reshape(-1,1).sqrt())          


In [23]:

sample = 1000
theta_in = torch.rand(sample,1)*2*np.pi
r_in = torch.rand(sample,1).sqrt()*0.99#*(9/20-1/9*torch.cos(5*theta_in))*0.90
y0_in = torch.cos(theta_in)
y1_in = torch.sin(theta_in)
y_in = torch.cat((y0_in,y1_in),1)*r_in
y_out = torch.cat((y0_in,y1_in),1)/r_in
G2_in = torch.zeros(sample,int_sample_num)
'''for i in range(sample):
    for j in range(int_sample_num):
        d = int_x[j,:]-y_in[i,:]
        r0 = d.norm()
        G2_in[i,j] = -1/(2*np.pi)*((d*int_n[j,:]).sum())/(r0*r0)
'''
for i in range(sample):
        d = int_x-y_in[i,:]
        r0 = d.norm(dim=1)
        G2_in[i,:] = -1/(2*np.pi)*((d*int_n).sum(axis=1))/(r0*r0)

In [12]:
file_handle=open('loss50_160.txt',mode='w')
for i in range(Epoc):
    file_handle.write(str(loss_all[i])+'\n')

In [24]:
mynet2 = mynet1.cpu()
device1 = torch.device('cpu')
M0 = 500
r1 = torch.rand(M0,1).sqrt()
theta1 = torch.rand(M0,1)*2*np.pi
x0,x1 = torch.cos(theta1),torch.sin(theta1)
x_in = torch.cat((x0,x1),1)*r1
x_out = torch.cat((x0,x1),1)/r1

int_xy_in = torch.zeros(M0*int_sample_num,4)
int_xy_bd = torch.zeros(M0*int_sample_num,4)
#A2_c = A2.to(device1)
in_u = torch.zeros(M0*sample,1)
g0 = torch.zeros(M0*sample,1)
bd_u = torch.zeros(M0*int_sample_num,1)
u_green = torch.zeros(M0*sample,1)
u_green_bd = torch.zeros(M0*int_sample_num,1)
for i in tqdm(range(M0)):
    int_xy_in[i*int_sample_num:(i+1)*int_sample_num,0:2] = x_in[i,:]
    int_xy_in[i*int_sample_num:(i+1)*int_sample_num,2:4] = int_x
    d0 = ((y_out-x_in[i,:])**2).sum(axis=1).sqrt().reshape(-1,1)
    in_u[i*sample:(i+1)*sample] = torch.log((y_in**2).sum(axis=1).sqrt().reshape(-1,1)*d0)/(2*np.pi) 
    g0[i*sample:(i+1)*sample] = torch.log(((y_in-x_in[i,:])**2).sum(axis=1).sqrt().reshape(-1,1))/(2*np.pi) 
    in_h = mynet2(int_xy_in[i*int_sample_num:(i+1)*int_sample_num,:])
    u_green[i*sample:(i+1)*sample] = G2_in@in_h/int_sample_num*np.pi*2
    #u_green_bd[i*int_sample_num:(i+1)*int_sample_num] = -A2_c@((in_h))

bd_u = (torch.log(((int_xy_in[:,0:2]-int_xy_in[:,2:4])**2).sum(axis=1).sqrt())/(2*np.pi)).reshape(-1,1)
    
#print(in_u.reshape(1,-1))
#print(u_green.reshape(1,-1))
#print(x_in)
print((u_green-in_u).size()) 
print((u_green-in_u).norm()/(in_u).norm())  
#print((u_green_bd-bd_u).norm()/(bd_u).norm()) 
print((u_green-in_u).norm()/(in_u-g0).norm()*10)  
#print((u_green-in_u).norm()/(in_u+g0).norm())  


100%|██████████| 500/500 [00:04<00:00, 115.53it/s]

torch.Size([500000, 1])
tensor(0.0084, grad_fn=<DivBackward0>)
tensor(0.0714, grad_fn=<MulBackward0>)
